In [2]:
import os
os.getcwd()

'/Users/kjh/Documents/geostat3_e9'

## Find Unique $(i, t)$ Entities

### E9

In [5]:
import pandas as pd
from pathlib import Path

# 파일 경로
path = Path("/Users/kjh/Documents/geostat3_e9/E9_panel_sgg_aggregated.csv")  # 필요시 교체

# 1) 데이터 로드
df = pd.read_csv(path)

# 2) 컬럼명 소문자/공백 정리
df.columns = df.columns.str.strip().str.lower()

# 3) 후보 컬럼 자동 매핑
ALIAS = {
    "sido":    ["sido", "시도", "광역시도", "province", "sido_nm", "sido_name", "sido_name_kor"],
    "sigungu": ["sigungu", "시군구", "sgg", "시군구명", "sigungu_nm", "sigungu_name"],
    "year":    ["year", "연도", "yyyy"],
    "date":    ["date", "날짜", "ym", "yyyymm", "연월"]
}

def pick(col_aliases):
    for c in col_aliases:
        if c in df.columns:
            return c
    return None

sido_col    = pick(ALIAS["sido"])
sigungu_col = pick(ALIAS["sigungu"])
year_col    = pick(ALIAS["year"])
date_col    = pick(ALIAS["date"])  # 연도 파생용(없으면 None)

if sido_col is None or sigungu_col is None:
    raise KeyError("시도/시군구 컬럼을 찾지 못했습니다. 컬럼명을 확인해주세요.")

# 4) year 파생 (year가 없고 date/yyyymm만 있을 때)
if year_col is None:
    if date_col is None:
        raise KeyError("연도(year) 또는 연월(date/yyyymm) 관련 컬럼을 찾지 못했습니다.")
    # 문자열 변환 후 앞 4자리로 연도 추출
    year_series = pd.to_datetime(
        df[date_col].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6],  # 202001 같은 형태 유도
        format="%Y%m", errors="coerce"
    ).dt.year
else:
    # 숫자/문자 혼재 대비하여 깔끔한 연도로 정규화
    year_series = pd.to_numeric(df[year_col], errors="coerce")
    if year_series.isna().all() and date_col is not None:
        year_series = pd.to_datetime(
            df[date_col].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6],
            format="%Y%m", errors="coerce"
        ).dt.year

if year_series.isna().any():
    # 불가피할 때만 남김. 필요시 dropna()로 제거 가능
    pass

# 5) 유일 조합 테이블 생성
uniq = (
    pd.DataFrame({
        "sido": df[sido_col].astype(str).str.strip(),
        "sigungu": df[sigungu_col].astype(str).str.strip(),
        "year": year_series.astype("Int64")  # 결측 허용 정수
    })
    .dropna(subset=["sido", "sigungu", "year"])
    .drop_duplicates()
    .sort_values(["sido", "sigungu", "year"])
    .reset_index(drop=True)
)

# 6) 저장(선택)
uniq.to_csv("unique_e9.csv", index=False, encoding="utf-8-sig")

uniq.head()


,sido,sigungu,year
0,강원도,강릉시,2009
1,강원도,강릉시,2010
2,강원도,강릉시,2011
3,강원도,강릉시,2012
4,강원도,강릉시,2013


### Trade

In [ ]:
import re
import pandas as pd
from pathlib import Path

# 파일 경로
path = Path("/Users/kjh/Documents/geostat3_e9/trade_panel.csv")

# 1) 데이터 로드
df = pd.read_csv(path)

# 2) 컬럼명 소문자/공백 정리
df.columns = df.columns.str.strip().str.lower()

# 3) 후보 컬럼 자동 매핑
ALIAS = {
    "sido":    ["sido", "시도", "광역시도", "province", "sido_nm", "sido_name", "sido_name_kor"],
    "sigungu": ["sigungu", "시군구", "sgg", "시군구명", "sigungu_nm", "sigungu_name"],
    "region":  ["region", "지역", "행정구역", "행정구역명", "시도_시군구"],
    "year":    ["year", "연도", "yyyy"],
    "date":    ["date", "날짜", "ym", "yyyymm", "연월"]
}

def pick(cols, in_df):
    for c in cols:
        if c in in_df.columns:
            return c
    return None

sido_col    = pick(ALIAS["sido"], df)
sigungu_col = pick(ALIAS["sigungu"], df)
region_col  = pick(ALIAS["region"], df)
year_col    = pick(ALIAS["year"], df)
date_col    = pick(ALIAS["date"], df)

# ---------- (A) region -> (sido, sigungu) 분리 ----------
# 공식/변경 포함 시도명 후보(긴 것 먼저 매칭)
SIDO_NAMES = [
    "세종특별자치시", "제주특별자치도", "강원특별자치도", "전북특별자치도",
    "서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시",
    "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도"
]
# 영문/축약 대응(있다면)
SIDO_EN_MAP = {
    "seoul": "서울특별시", "busan": "부산광역시", "daegu": "대구광역시", "incheon": "인천광역시",
    "gwangju": "광주광역시", "daejeon": "대전광역시", "ulsan": "울산광역시", "sejong": "세종특별자치시",
    "gyeonggi": "경기도", "gangwon": "강원특별자치도", "chungbuk": "충청북도", "chungnam": "충청남도",
    "jeonbuk": "전북특별자치도", "jeonnam": "전라남도", "gyeongbuk": "경상북도", "gyeongnam": "경상남도",
    "jeju": "제주특별자치도"
}

def clean_region(s):
    if pd.isna(s):
        return None
    s = str(s)
    s = re.sub(r"[\(\[\{].*?[\)\]\}]", "", s)  # 괄호 내용 제거
    s = s.replace("/", " ").replace("-", " ").replace(",", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def split_region(s):
    """ region 문자열에서 (시도, 시군구) 추출 """
    if not s:
        return (None, None)
    raw = clean_region(s)

    # 1) 쉼표/슬래시로 이미 구분된 케이스 처리 후 재시도
    for sep in [",", "/", "-"]:
        if sep in str(s):
            parts = [p.strip() for p in str(s).split(sep)]
            if len(parts) >= 2:
                return split_region(" ".join(parts))  # 공백 기반 재처리

    # 2) 시도명 prefix 매칭(가장 긴 시도명 우선)
    for sido_name in sorted(SIDO_NAMES, key=len, reverse=True):
        if raw.startswith(sido_name):
            rest = raw[len(sido_name):].strip()
            return (sido_name, rest if rest else None)

    # 3) 영문/로마자 시도명 시작하는 경우
    first_tok = raw.split(" ")[0].lower()
    if first_tok in SIDO_EN_MAP:
        sido_name = SIDO_EN_MAP[first_tok]
        rest = raw[len(first_tok):].strip()
        # 영문 토큰 제거를 위해 첫 토큰 제외
        rest = " ".join(raw.split(" ")[1:]).strip() or None
        return (sido_name, rest)

    # 4) fallback: 첫 토큰을 시도, 나머지를 시군구로
    toks = raw.split(" ")
    if len(toks) == 1:
        return (toks[0], None)
    return (toks[0], " ".join(toks[1:]))

# region이 있고(또는 시도/시군구 중 하나라도 없으면) region에서 파생
if region_col is not None and (sido_col is None or sigungu_col is None):
    tmp = df[region_col].map(clean_region)
    parsed = tmp.map(split_region)
    df["_sido_from_region"]    = parsed.map(lambda x: x[0])
    df["_sigungu_from_region"] = parsed.map(lambda x: x[1])
    # 우선순위: 기존 컬럼 > region 파생
    if sido_col is None:
        sido_col = "_sido_from_region"
    if sigungu_col is None:
        sigungu_col = "_sigungu_from_region"

# 최종 체크
if sido_col is None or sigungu_col is None:
    raise KeyError("시도/시군구 컬럼을 찾지 못했습니다. (region 분해 실패 가능)")

# ---------- (B) 연도 파생 ----------
if year_col is None:
    if date_col is None:
        raise KeyError("연도(year) 또는 연월(date/yyyymm) 관련 컬럼을 찾지 못했습니다.")
    year_series = pd.to_datetime(
        df[date_col].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6],
        format="%Y%m", errors="coerce"
    ).dt.year
else:
    year_series = pd.to_numeric(df[year_col], errors="coerce")
    if year_series.isna().all() and date_col is not None:
        year_series = pd.to_datetime(
            df[date_col].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6],
            format="%Y%m", errors="coerce"
        ).dt.year

# ---------- (C) 유일 조합 테이블 ----------
uniq = (
    pd.DataFrame({
        "sido": df[sido_col].astype(str).str.strip(),
        "sigungu": df[sigungu_col].astype(str).str.strip(),
        "year": year_series.astype("Int64")
    })
    .dropna(subset=["sido", "year"])  # sigungu가 없는 행은 남기고 싶으면 여기서 제외하지 말 것
    .assign(sigungu=lambda x: x["sigungu"].replace({"": pd.NA}))
    .drop_duplicates()
    .sort_values(["sido", "sigungu", "year"], na_position="last")
    .reset_index(drop=True)
)

# 저장
uniq.to_csv("unique_trade.csv", index=False, encoding="utf-8-sig")

uniq.head()


,sido,sigungu,year
0,강원특별자치도,강릉시,2008
1,강원특별자치도,강릉시,2009
2,강원특별자치도,강릉시,2010
3,강원특별자치도,강릉시,2011
4,강원특별자치도,강릉시,2012


In [ ]:
# import re
# import pandas as pd
# from pathlib import Path

# # 파일 경로
# path = Path("/Users/kjh/Documents/geostat3_e9/manufacturing_production_index.csv")

# # 1) 데이터 로드
# df = pd.read_csv(path)

# # 2) 컬럼명 소문자/공백 정리
# df.columns = df.columns.str.strip().str.lower()

# # 3) 후보 컬럼 자동 매핑
# ALIAS = {
#     "sido":    ["sido", "시도", "광역시도", "province", "sido_nm", "sido_name", "sido_name_kor"],
#     "sigungu": ["sigungu", "시군구", "sgg", "시군구명", "sigungu_nm", "sigungu_name"],
#     "region":  ["region", "지역", "행정구역", "행정구역명", "시도_시군구"],
#     "year":    ["year", "연도", "yyyy"],
#     "date":    ["date", "날짜", "ym", "yyyymm", "연월"]
# }

# def pick(cols, in_df):
#     for c in cols:
#         if c in in_df.columns:
#             return c
#     return None

# sido_col    = pick(ALIAS["sido"], df)
# sigungu_col = pick(ALIAS["sigungu"], df)
# region_col  = pick(ALIAS["region"], df)
# year_col    = pick(ALIAS["year"], df)
# date_col    = pick(ALIAS["date"], df)

# # ---------- (A) region -> (sido, sigungu) 분리 ----------
# # 공식/변경 포함 시도명 후보(긴 것 먼저 매칭)
# SIDO_NAMES = [
#     "세종특별자치시", "제주특별자치도", "강원특별자치도", "전북특별자치도",
#     "서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시",
#     "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도"
# ]
# # 영문/축약 대응(있다면)
# SIDO_EN_MAP = {
#     "seoul": "서울특별시", "busan": "부산광역시", "daegu": "대구광역시", "incheon": "인천광역시",
#     "gwangju": "광주광역시", "daejeon": "대전광역시", "ulsan": "울산광역시", "sejong": "세종특별자치시",
#     "gyeonggi": "경기도", "gangwon": "강원특별자치도", "chungbuk": "충청북도", "chungnam": "충청남도",
#     "jeonbuk": "전북특별자치도", "jeonnam": "전라남도", "gyeongbuk": "경상북도", "gyeongnam": "경상남도",
#     "jeju": "제주특별자치도"
# }

# def clean_region(s):
#     if pd.isna(s):
#         return None
#     s = str(s)
#     s = re.sub(r"[\(\[\{].*?[\)\]\}]", "", s)  # 괄호 내용 제거
#     s = s.replace("/", " ").replace("-", " ").replace(",", " ")
#     s = re.sub(r"\s+", " ", s).strip()
#     return s

# def split_region(s):
#     """ region 문자열에서 (시도, 시군구) 추출 """
#     if not s:
#         return (None, None)
#     raw = clean_region(s)

#     # 1) 쉼표/슬래시로 이미 구분된 케이스 처리 후 재시도
#     for sep in [",", "/", "-"]:
#         if sep in str(s):
#             parts = [p.strip() for p in str(s).split(sep)]
#             if len(parts) >= 2:
#                 return split_region(" ".join(parts))  # 공백 기반 재처리

#     # 2) 시도명 prefix 매칭(가장 긴 시도명 우선)
#     for sido_name in sorted(SIDO_NAMES, key=len, reverse=True):
#         if raw.startswith(sido_name):
#             rest = raw[len(sido_name):].strip()
#             return (sido_name, rest if rest else None)

#     # 3) 영문/로마자 시도명 시작하는 경우
#     first_tok = raw.split(" ")[0].lower()
#     if first_tok in SIDO_EN_MAP:
#         sido_name = SIDO_EN_MAP[first_tok]
#         rest = raw[len(first_tok):].strip()
#         # 영문 토큰 제거를 위해 첫 토큰 제외
#         rest = " ".join(raw.split(" ")[1:]).strip() or None
#         return (sido_name, rest)

#     # 4) fallback: 첫 토큰을 시도, 나머지를 시군구로
#     toks = raw.split(" ")
#     if len(toks) == 1:
#         return (toks[0], None)
#     return (toks[0], " ".join(toks[1:]))

# # region이 있고(또는 시도/시군구 중 하나라도 없으면) region에서 파생
# if region_col is not None and (sido_col is None or sigungu_col is None):
#     tmp = df[region_col].map(clean_region)
#     parsed = tmp.map(split_region)
#     df["_sido_from_region"]    = parsed.map(lambda x: x[0])
#     df["_sigungu_from_region"] = parsed.map(lambda x: x[1])
#     # 우선순위: 기존 컬럼 > region 파생
#     if sido_col is None:
#         sido_col = "_sido_from_region"
#     if sigungu_col is None:
#         sigungu_col = "_sigungu_from_region"

# # 최종 체크
# if sido_col is None or sigungu_col is None:
#     raise KeyError("시도/시군구 컬럼을 찾지 못했습니다. (region 분해 실패 가능)")

# # ---------- (B) 연도 파생 ----------
# if year_col is None:
#     if date_col is None:
#         raise KeyError("연도(year) 또는 연월(date/yyyymm) 관련 컬럼을 찾지 못했습니다.")
#     year_series = pd.to_datetime(
#         df[date_col].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6],
#         format="%Y%m", errors="coerce"
#     ).dt.year
# else:
#     year_series = pd.to_numeric(df[year_col], errors="coerce")
#     if year_series.isna().all() and date_col is not None:
#         year_series = pd.to_datetime(
#             df[date_col].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6],
#             format="%Y%m", errors="coerce"
#         ).dt.year

# # ---------- (C) 유일 조합 테이블 ----------
# uniq = (
#     pd.DataFrame({
#         "sido": df[sido_col].astype(str).str.strip(),
#         "sigungu": df[sigungu_col].astype(str).str.strip(),
#         "year": year_series.astype("Int64")
#     })
#     .dropna(subset=["sido", "year"])  # sigungu가 없는 행은 남기고 싶으면 여기서 제외하지 말 것
#     .assign(sigungu=lambda x: x["sigungu"].replace({"": pd.NA}))
#     .drop_duplicates()
#     .sort_values(["sido", "sigungu", "year"], na_position="last")
#     .reset_index(drop=True)
# )

# # 저장
# uniq.to_csv("unique_manufacturing.csv", index=False, encoding="utf-8-sig")

# uniq.head()


,sido,sigungu,year
0,강원도,None,2008
1,강원도,None,2009
2,강원도,None,2010
3,강원도,None,2011
4,강원도,None,2012


### Population

In [30]:
# -*- coding: utf-8 -*-
"""
unique_pop 생성 (검증된 컬럼명 사용)
- 입력 : /Users/kjh/Documents/geostat3_e9/population_panel_sgg_aggregated.csv
- 출력 : unique_pop.csv  (컬럼: sido, sigungu, year)
- 비고 : 광역·특별(자치)시는 '시도명 + 구/군' 형태가 이미 들어 있으므로
        남구/서구/중구 등의 중복 문제 없이 안전.
"""

import pandas as pd
from pathlib import Path

# 경로 설정 (필요시 수정)
IN_PATH  = Path("/Users/kjh/Documents/geostat3_e9/population_panel_sgg_aggregated.csv")
OUT_PATH = Path("unique_pop.csv")  # 현재 작업 디렉토리에 저장

# 1) 로드
df = pd.read_csv(IN_PATH)

# 2) 필수 컬럼 존재 확인 (실제 파일 기준)
required = {"sido_name", "sigungu", "year"}
missing = required - set(df.columns)
if missing:
    raise KeyError(f"필수 컬럼 누락: {missing}  (파일 컬럼: {list(df.columns)})")

# 3) 정리 & 유일 조합
uniq = (
    df.loc[:, ["sido_name", "sigungu", "year"]]
      .rename(columns={"sido_name": "sido"})
      .assign(
          sido=lambda x: x["sido"].astype(str).str.strip(),
          sigungu=lambda x: x["sigungu"].astype(str).str.strip(),
          year=lambda x: pd.to_numeric(x["year"], errors="coerce").astype("Int64")
      )
      .dropna(subset=["sido", "sigungu", "year"])
      .drop_duplicates()
      .sort_values(["sido", "sigungu", "year"])
      .reset_index(drop=True)
)

# 4) 저장
uniq.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

print(f"[OK] unique_pop.csv saved: {OUT_PATH}  rows={len(uniq):,}")
print(uniq.head(10))


[OK] unique_pop.csv saved: unique_pop.csv  rows=4,211
      sido sigungu  year
0  강원특별자치도     강릉시  2008
1  강원특별자치도     강릉시  2009
2  강원특별자치도     강릉시  2010
3  강원특별자치도     강릉시  2011
4  강원특별자치도     강릉시  2012
5  강원특별자치도     강릉시  2013
6  강원특별자치도     강릉시  2014
7  강원특별자치도     강릉시  2015
8  강원특별자치도     강릉시  2016
9  강원특별자치도     강릉시  2017


## Construct Matching Table

In [4]:
# -*- coding: utf-8 -*-
"""
매칭 테이블 생성 (시군구 × 연-월 조인키)
- 코드북: adm_code_general_2024.xlsx (sheet='Transform')
- 소스: unique_e9.csv, unique_pop.csv, unique_trade.csv, (옵션) unique_manufacturing.csv

출력:
  - matching_table_join_keys.csv
  - unmatched_rows_<src>.csv

포인트
- 시도+시군구 복합키(join_key)로 1차 매칭
- Transform 시트의 대체명칭 열들을 이용해 code_key를 확장
- 강원/전북 특별자치도 등 시도 별칭 정규화
- (중요) 소스의 sigungu가 '서울특별시 강남구' 같은 형태면 시도 접두어 제거
- 소스/코드 모두 숫자 프리픽스('11000 서울특별시') 제거
"""

from pathlib import Path
import pandas as pd
import numpy as np
import unicodedata
import re

# -----------------------------
# 경로 & 소스
# -----------------------------
BASE = Path("/Users/kjh/Documents/geostat3_e9")
CODEBOOK_XLSX = BASE / "adm_code_general_2024.xlsx"
SOURCES = {
    "e9":   BASE / "unique_e9.csv",
    "pop":  BASE / "unique_pop.csv",
    "trade":BASE / "unique_trade.csv",
    # "manufacturing": BASE / "unique_manufacturing.csv",
}

# (옵션) 연도 필터
YEAR_START = 2009   # 예: 2009
YEAR_END   = None   # 예: 2025

# -----------------------------
# 유틸
# -----------------------------
def normalize_korean(text: str) -> str:
    """공백/괄호 제거 + NFC 정규화"""
    if pd.isna(text):
        return text
    s = unicodedata.normalize("NFC", str(text)).strip()
    s = re.sub(r"\s+", "", s)
    s = re.sub(r"[()（）\[\]]", "", s)
    return s

SIDO_ALIAS = {
    normalize_korean("강원도"):          normalize_korean("강원특별자치도"),
    normalize_korean("강원특별자치도"):  normalize_korean("강원특별자치도"),
    normalize_korean("전라북도"):        normalize_korean("전북특별자치도"),
    normalize_korean("전북특별자치도"):  normalize_korean("전북특별자치도"),
    normalize_korean("전북"):            normalize_korean("전북특별자치도"),
    normalize_korean("세종시"):          normalize_korean("세종특별자치시"),
    normalize_korean("세종특별자치시"):  normalize_korean("세종특별자치시"),
    normalize_korean("제주도"):          normalize_korean("제주특별자치도"),
    normalize_korean("제주특별자치도"):  normalize_korean("제주특별자치도"),
}

def canonicalize_sido(s: str) -> str:
    if pd.isna(s):
        return s
    s_norm = normalize_korean(s)
    return SIDO_ALIAS.get(s_norm, s_norm)

def clean_sido_name(x: str) -> str:
    """'001 서울특별시' → '서울특별시'"""
    if pd.isna(x):
        return x
    return re.sub(r"^\s*\d+\s*", "", str(x)).strip()

def digits(s):
    return re.sub(r"[^0-9]", "", str(s)) if pd.notna(s) else ""

def coerce_yyyymm(df: pd.DataFrame) -> pd.Series:
    """date/yyyymm/ym/연월 → 6자리 추출, 없으면 year+month, 없으면 year*100+01"""
    cols_lower = {c.lower(): c for c in df.columns}
    for k in ["date", "yyyymm", "ym", "연월", "year_month"]:
        if k in cols_lower:
            s = df[cols_lower[k]].astype(str).str.replace(r"[^0-9]", "", regex=True).str[:6]
            s = s.replace("", np.nan)
            return s.astype("Int64")
    ycol = next((cols_lower[k] for k in ["year", "연도", "yyyy", "yr"] if k in cols_lower), None)
    mcol = next((cols_lower[k] for k in ["month", "월", "mm", "mnth"] if k in cols_lower), None)
    if ycol and mcol:
        yy = pd.to_numeric(df[ycol], errors="coerce").astype("Int64")
        mm = pd.to_numeric(df[mcol], errors="coerce").astype("Int64")
        return (yy * 100 + mm).astype("Int64")
    if ycol:
        yy = pd.to_numeric(df[ycol], errors="coerce").astype("Int64")
        return (yy * 100 + 1).astype("Int64")
    raise ValueError("연-월(yyyymm) 정보를 유추할 수 없습니다.")

def strip_leading_digits(x: str) -> str:
    """앞 숫자 토큰 제거: '11000 서울특별시' → '서울특별시'"""
    if pd.isna(x):
        return x
    return re.sub(r"^\s*\d+\s*", "", str(x)).strip()

def strip_city_prefix(sigungu_norm: str, sido_norm: str) -> str:
    """
    '서울특별시강남구' → '강남구' (이미 normalize된 문자열 기준)
    시도명과 완전히 같은 경우(세종특별자치시 등)는 그대로 둠.
    """
    if pd.isna(sigungu_norm) or pd.isna(sido_norm):
        return sigungu_norm
    sgg = str(sigungu_norm); sido = str(sido_norm)
    if sgg == sido:
        return sgg
    return sgg[len(sido):] if sgg.startswith(sido) else sgg

# -----------------------------
# 코드북 Transform → code_key 확장
# -----------------------------
code = pd.read_excel(CODEBOOK_XLSX, sheet_name="Transform", dtype=str).rename(columns=str.strip)

# 기본 키(주명칭)
base = code[["SIDO","SGGNM_ADM","SGGCD_ADM_2024","SGIS_2019"]].copy()
base["sido_name"]    = base["SIDO"].map(clean_sido_name)
base["sido_norm"]    = base["sido_name"].map(normalize_korean).map(canonicalize_sido)
base["sigungu_name"] = base["SGGNM_ADM"].astype(str).map(strip_leading_digits)
base["sigungu_norm"] = base["sigungu_name"].map(normalize_korean)
base["sgg_code10"]   = base["SGGCD_ADM_2024"].map(digits)
base["sgis7_2019"]   = base["SGIS_2019"].map(digits)

# 대체 명칭을 키로 확장
ALT_SIG_COLS = [
    "SIGUNGU_NM","SIGUNGU","SIGUNGU_SUB_NM","SIGUNGU_PSEUDO","SIGUNGU_PSEUDO0",
    "SGGNM_DOJ","SGGNM_DCNM"
]
alts = []
for c in ALT_SIG_COLS:
    if c in code.columns:
        t = code[["SIDO", c, "SGGCD_ADM_2024", "SGIS_2019"]].copy()
        t = t.rename(columns={c: "alt_sigungu"})
        t["sido_name"]    = t["SIDO"].map(clean_sido_name)
        t["sido_norm"]    = t["sido_name"].map(normalize_korean).map(canonicalize_sido)
        t["sigungu_name"] = t["alt_sigungu"].astype(str).map(strip_leading_digits)
        t["sigungu_norm"] = t["sigungu_name"].map(normalize_korean)
        t["sgg_code10"]   = t["SGGCD_ADM_2024"].map(digits)
        t["sgis7_2019"]   = t["SGIS_2019"].map(digits)
        alts.append(t[["sido_norm","sigungu_norm","sido_name","sigungu_name","sgg_code10","sgis7_2019"]])

code_key = pd.concat(
    [base[["sido_norm","sigungu_norm","sido_name","sigungu_name","sgg_code10","sgis7_2019"]]] + alts,
    ignore_index=True
)
# 키 구성
code_key = code_key.dropna(subset=["sigungu_norm"])
code_key["join_key"] = (code_key["sido_norm"] + "::" + code_key["sigungu_norm"]).astype("string")
code_key = code_key.drop_duplicates(subset=["join_key"]).reset_index(drop=True)

# 전국에서 시군구명이 유일한 경우(보조 매칭용)
unique_sigungu = code_key.groupby("sigungu_norm")["sgg_code10"].nunique().rename("n_code").reset_index()
sigungu_unique_only = (
    code_key.merge(unique_sigungu, on="sigungu_norm", how="left")
            .query("n_code == 1")[["sigungu_norm","sgg_code10","sgis7_2019","sido_name","sigungu_name"]]
            .drop_duplicates()
)

# -----------------------------
# 소스 로드 & 매칭
# -----------------------------
def load_source(src_name: str, path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype=str)
    df.columns = df.columns.str.strip()

    if "sigungu" not in df.columns:
        raise ValueError(f"[{src_name}] 'sigungu' 컬럼이 필요합니다.")
    if "sido" not in df.columns:
        df["sido"] = np.nan

    # 숫자 프리픽스 제거(혼용 대비), 정규화, 시도 별칭, 시도 접두어 제거
    sido_raw    = df["sido"].map(strip_leading_digits).astype("string")
    sigungu_raw = df["sigungu"].map(strip_leading_digits).astype("string")

    out = pd.DataFrame({
        "sido_raw":    sido_raw,
        "sigungu_raw": sigungu_raw,
    })
    out["sido_norm"]    = out["sido_raw"].map(normalize_korean).map(canonicalize_sido).astype("string")
    out["sigungu_norm"] = out["sigungu_raw"].map(normalize_korean).astype("string")
    # 특별시/광역시 접두어 제거 (예: '서울특별시강남구' → '강남구')
    out["sigungu_norm"] = pd.Series(
        [strip_city_prefix(sgg, sido) for sgg, sido in zip(out["sigungu_norm"], out["sido_norm"])],
        dtype="string"
    )

    out["join_key"] = (out["sido_norm"] + "::" + out["sigungu_norm"]).astype("string")
    out["yyyymm"]   = coerce_yyyymm(df).astype("Int64")
    out["src"]      = src_name

    if YEAR_START is not None or YEAR_END is not None:
        yy = (out["yyyymm"] // 100).astype("Int64")
        if YEAR_START is not None:
            out = out[yy >= YEAR_START]
        if YEAR_END is not None:
            out = out[yy <= YEAR_END]

    return out.drop_duplicates(subset=["join_key","yyyymm"]).reset_index(drop=True)

loaded = {src: load_source(src, path) for src, path in SOURCES.items()}

matched_frames = {}
unmatched_frames = {}

for src, df in loaded.items():
    # 1차: 복합키로 정확 매칭
    m1 = df.merge(
        code_key[["join_key","sgg_code10","sgis7_2019","sido_name","sigungu_name"]],
        on="join_key", how="left"
    )

    # 2차: 1차 실패분 → 전국 유일 시군구명으로 보조 매칭
    need_fb = m1[m1["sgg_code10"].isna()].copy()
    if not need_fb.empty:
        fb = need_fb.merge(sigungu_unique_only, on="sigungu_norm", how="left", suffixes=("", "_fb"))
        for col in ["sgg_code10","sgis7_2019","sido_name","sigungu_name"]:
            m1.loc[fb.index, col] = m1.loc[fb.index, col].fillna(fb[col])

    matched   = m1[m1["sgg_code10"].notna()].copy()
    unmatched = m1[m1["sgg_code10"].isna()].copy()

    matched_frames[src] = matched[
        ["sgg_code10","sgis7_2019","yyyymm","sido_raw","sigungu_raw","sido_name","sigungu_name"]
    ].rename(columns={"sido_raw": f"sido_{src}", "sigungu_raw": f"sigungu_{src}"})

    unmatched_frames[src] = (
        unmatched[["yyyymm","sido_raw","sigungu_raw"]]
        .rename(columns={"sido_raw": "sido", "sigungu_raw": "sigungu"})
    )

# -----------------------------
# 마스터(와이드) 구성 & 저장
# -----------------------------
master = None
for src, mf in matched_frames.items():
    master = mf.copy() if master is None else master.merge(
        mf,
        on=["sgg_code10","sgis7_2019","yyyymm","sido_name","sigungu_name"],
        how="outer"
    )

for src in matched_frames.keys():
    master[f"present_{src}"] = master[f"sigungu_{src}"].notna().astype(int)

master = master.sort_values(["sgg_code10","yyyymm"]).reset_index(drop=True)

out_master = BASE / "matching_table_join_keys.csv"
master.to_csv(out_master, index=False, encoding="utf-8-sig")

for src, uf in unmatched_frames.items():
    out_unmatched = BASE / f"unmatched_rows_{src}.csv"
    (uf.sort_values(["yyyymm","sigungu"])
       .drop_duplicates()
       .to_csv(out_unmatched, index=False, encoding="utf-8-sig"))

print(f"[OK] code_key rows={len(code_key):,}, unique join_key={code_key['join_key'].nunique():,}")
print(f"[OK] Master saved: {out_master} rows={len(master):,} cols={len(master.columns)}")
for src, uf in unmatched_frames.items():
    print(f"[OK] Unmatched ({src}): {len(uf):,} -> {BASE / f'unmatched_rows_{src}.csv'}")


[OK] code_key rows=602, unique join_key=602
[OK] Master saved: /Users/kjh/Documents/geostat3_e9/matching_table_join_keys.csv rows=3,926 cols=14
[OK] Unmatched (e9): 21 -> /Users/kjh/Documents/geostat3_e9/unmatched_rows_e9.csv
[OK] Unmatched (pop): 82 -> /Users/kjh/Documents/geostat3_e9/unmatched_rows_pop.csv
[OK] Unmatched (trade): 53 -> /Users/kjh/Documents/geostat3_e9/unmatched_rows_trade.csv


### 특이값 필터링

In [9]:
# Using the actual columns from matching_table_join_keys.csv to compute the requested outputs
import pandas as pd
import numpy as np
from collections import Counter

path = "/Users/kjh/Documents/geostat3_e9/matching_table_join_keys.csv"
master = pd.read_csv(path)

# Normalize year from yyyymm (use first 4 digits)
master = master.copy()
master['year'] = master['yyyymm'].astype(str).str.slice(0,4).astype(int)

in_range = master['year'].between(2009, 2025)
code_col = 'sgg_code10'
present_cols = ['present_e9', 'present_pop', 'present_trade']

# Build full grid code × year
codes = master.loc[in_range, code_col].dropna().unique()
years = np.arange(2009, 2026, dtype=int)
full_grid = pd.MultiIndex.from_product([codes, years], names=[code_col, 'year']).to_frame(index=False)

# Observed (code, year) pairs
seen = master.loc[in_range, [code_col, 'year']].drop_duplicates()

# Missing pairs
missing_pairs = (
    full_grid
    .merge(seen, on=[code_col, 'year'], how='left', indicator=True)
    .query("_merge == 'left_only'")
    .drop(columns="_merge")
    .sort_values([code_col, 'year'])
    .reset_index(drop=True)
)

# Attach names for missing_pairs:
# 1) Try exact (code, year) mapping from master if exists
name_map_cy = (
    master.loc[in_range, [code_col, 'year', 'sido_name', 'sigungu_name']]
    .dropna(subset=[code_col, 'year'])
    .drop_duplicates()
)

missing_pairs = missing_pairs.merge(
    name_map_cy, on=[code_col, 'year'], how='left'
)

# 2) For any remaining NaNs, fill by most frequent name per code across available years
name_map_code = (
    master.loc[in_range, [code_col, 'sido_name', 'sigungu_name']]
    .dropna(subset=[code_col])
)

def mode_or_first(series):
    if series.isna().all():
        return pd.NA
    counts = Counter(series.dropna())
    # Return the most common value; deterministic tie-break by value
    most_common = sorted(counts.items(), key=lambda x: (-x[1], str(x[0])))[0][0]
    return most_common

fallback = name_map_code.groupby(code_col).agg({
    'sido_name': mode_or_first,
    'sigungu_name': mode_or_first
}).reset_index().rename(columns={
    'sido_name': 'sido_name_fallback',
    'sigungu_name': 'sigungu_name_fallback'
})

missing_pairs = missing_pairs.merge(fallback, on=code_col, how='left')

for col in ['sido_name', 'sigungu_name']:
    missing_pairs[col] = missing_pairs[col].fillna(missing_pairs[f"{col}_fallback"])
    missing_pairs.drop(columns=[f"{col}_fallback"], inplace=True)

# Reorder columns
missing_pairs = missing_pairs[[code_col, 'sido_name', 'sigungu_name', 'year']]

# Rows where present_* not all 1 (keep names)
not_all_one = (
    master.loc[in_range]
    .loc[lambda d: d[present_cols].ne(1).any(axis=1),
         [code_col, 'sido_name', 'sigungu_name', 'yyyymm', 'year'] + present_cols]
    .sort_values([code_col, 'year'])
    .reset_index(drop=True)
)

# Save outputs for download and also preview in UI
out1 = "/Users/kjh/Documents/geostat3_e9/missing_code_year_pairs_2009_2025.csv"
out2 = "/Users/kjh/Documents/geostat3_e9/not_all_one_present_rows_2009_2025.csv"
missing_pairs.to_csv(out1, index=False, encoding="utf-8-sig")
not_all_one.to_csv(out2, index=False, encoding="utf-8-sig")


- 당진: 군+시 합산
- 여주: 군/시 -> 시
- 포천: 군/시 -> 시
- 연기: 연기/세종시 -> 세종시
- 군위: 경북/대구 -> 대구
- 청원: 청주와 합산
- 인천 남구: 남구/미추홀구 -> 미추홀구
- 마산/진해: 창원과 합산

___

## Join

In [22]:
# Rebuild the monthly joined panel with **time-agnostic** canonical mapping,
# as requested: always map historical/future records to the same canonical unit.
#  - 2+→1 cases (Dangjin, Cheongju, Changwon) → **sum values** into the canonical unit
#  - 1→1 cases (Yeoju, Pocheon, Sejong, Michuhol, Gunwi) → **rename only**
#
# Inputs (uploaded):
#   /mnt/data/matching_table_join_keys.csv
#   /mnt/data/E9_panel_sgg_aggregated.csv
#   /mnt/data/population_panel_sgg_aggregated.csv
#   /mnt/data/trade_panel.csv
# Output:
#   /mnt/data/panel_sgg_monthly_canonical_alltime.csv

import pandas as pd
import numpy as np
from pathlib import Path
import unicodedata, re

BASE = Path("/Users/kjh/Documents/geostat3_e9")
PATH_MT   = BASE / "matching_table_join_keys.csv"
PATH_E9   = BASE / "E9_panel_sgg_aggregated.csv"
PATH_POP  = BASE / "population_panel_sgg_aggregated.csv"
PATH_TR   = BASE / "trade_panel.csv"
OUT_PATH  = BASE / "panel_sgg_monthly.csv"

def nrm(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFC", str(s)).strip()
    s = re.sub(r"\s+","", s)
    s = re.sub(r"[()（）\[\]]","", s)
    return s

# Load matching table
mt = pd.read_csv(PATH_MT, dtype=str).rename(columns=str.strip)

# Build source-priority names (pop -> e9 -> trade) to tag sgg_code10 with a representative (sido, sigungu)
has_pop   = {"sido_pop","sigungu_pop"}.issubset(mt.columns)
has_e9    = {"sido_e9","sigungu_e9"}.issubset(mt.columns)
has_trade = {"sido_trade","sigungu_trade"}.issubset(mt.columns)

def pick_name(r):
    if has_pop   and pd.notna(r.get("sido_pop"))   and pd.notna(r.get("sigungu_pop")):   return r["sido_pop"], r["sigungu_pop"]
    if has_e9    and pd.notna(r.get("sido_e9"))    and pd.notna(r.get("sigungu_e9")):    return r["sido_e9"],  r["sigungu_e9"]
    if has_trade and pd.notna(r.get("sido_trade")) and pd.notna(r.get("sigungu_trade")): return r["sido_trade"], r["sigungu_trade"]
    return None, None

mt["src_sido"], mt["src_sigungu"] = zip(*mt.apply(pick_name, axis=1))
rep = (mt.dropna(subset=["sgg_code10","src_sido","src_sigungu"])
         .groupby("sgg_code10")[["src_sido","src_sigungu"]]
         .agg(lambda col: col.value_counts().index[0])
         .reset_index())

# Build (sido,sigungu) -> sgg_code10 maps by source
def most_frequent_code(df: pd.DataFrame, cs: str, cg: str) -> pd.DataFrame:
    if cs not in df.columns or cg not in df.columns:
        return pd.DataFrame(columns=["sido_norm","sigungu_norm","sgg_code10"])
    tmp = df[[cs, cg, "sgg_code10"]].dropna()
    if tmp.empty: 
        return pd.DataFrame(columns=["sido_norm","sigungu_norm","sgg_code10"])
    tmp["sido_norm"]    = tmp[cs].map(nrm)
    tmp["sigungu_norm"] = tmp[cg].map(nrm)
    mapdf = (tmp.groupby(["sido_norm","sigungu_norm"])["sgg_code10"]
                .agg(lambda s: s.value_counts().index[0]).reset_index())
    return mapdf[["sido_norm","sigungu_norm","sgg_code10"]].drop_duplicates()

map_e9    = most_frequent_code(mt, "sido_e9",   "sigungu_e9")
map_pop   = most_frequent_code(mt, "sido_pop",  "sigungu_pop")
map_trade = most_frequent_code(mt, "sido_trade","sigungu_trade")

# Time-agnostic canonical mapping rules
SET_DANGJIN  = {"당진군","당진시"}
SET_CHEONGJU = {"청원군","청주시","청원"}
SET_CHANGWON = {"창원시","마산시","진해시","마산","진해","마산합포구","마산회원구","진해구","성산구","의창구"}

SET_MICHHOL  = {"남구","미추홀구"}
SET_SEJONG   = {"연기","연기군","세종시"}
SET_GUNWI    = {"군위","군위군"}
SET_YEOJU    = {"여주군","여주시"}
SET_POCHEON  = {"포천군","포천시"}

def canon_name_alltime(sido, sgg):
    sdn = nrm(sido) if pd.notna(sido) else None
    sgn = nrm(sgg)  if pd.notna(sgg)  else None

    # 2+→1 (sum) groups: always map to canonical titles
    if sgn in SET_DANGJIN:   return "충청남도","당진시"
    if sgn in SET_CHEONGJU:  return "충청북도","청주시"
    if sgn in SET_CHANGWON:  return "경상남도","창원시"

    # 1→1 (rename only) groups: always map to canonical titles
    if (sgn in SET_SEJONG) or (sdn in {"세종특별자치시","세종시"}):
        return "세종특별자치시","세종시"
    if sdn == "인천광역시" and (sgn in SET_MICHHOL):
        return "인천광역시","미추홀구"
    if sgn in SET_GUNWI:
        return "대구광역시","군위군"
    if sgn in SET_YEOJU:
        return "경기도","여주시"
    if sgn in SET_POCHEON:
        return "경기도","포천시"

    # default: keep original
    return sido, sgg

# Load panels
e9 = pd.read_csv(PATH_E9, dtype=str).rename(columns={"value":"e9"})
e9["yyyymm"] = pd.to_numeric(e9["yyyymm"], errors="coerce").astype("Int64")
e9["sido_norm"]    = e9["sido"].map(nrm)
e9["sigungu_norm"] = e9["sigungu"].map(nrm)
e9 = e9.merge(map_e9, on=["sido_norm","sigungu_norm"], how="inner")
e9_agg = e9.groupby(["sgg_code10","yyyymm"], as_index=False)[["e9"]].sum()

pop = pd.read_csv(PATH_POP, dtype=str).rename(columns={"total":"population_total",
                                                       "male":"population_male",
                                                       "female":"population_female"})
pop["yyyymm"] = pd.to_numeric(pop["yyyymm"], errors="coerce").astype("Int64")
pop["sido_norm"]    = pop["sido_name"].map(nrm)
pop["sigungu_norm"] = pop["sigungu"].map(nrm)
pop = pop.merge(map_pop, on=["sido_norm","sigungu_norm"], how="inner")
pop_agg = pop.groupby(["sgg_code10","yyyymm"], as_index=False)[["population_total","population_male","population_female"]].sum()

tr = pd.read_csv(PATH_TR, dtype=str)
split = tr["region"].str.split(r"\s+", n=1, expand=True)
tr["sido"]    = split[0]
tr["sigungu"] = split[1]
tr["yyyymm"]  = (pd.to_numeric(tr["year"], errors="coerce")*100 + pd.to_numeric(tr["month"], errors="coerce")).astype("Int64")
tr = tr.rename(columns={"export_cnt":"trade_export_cnt",
                        "export_val":"trade_export_val",
                        "import_cnt":"trade_import_cnt",
                        "import_val":"trade_import_val"})
for c in ["trade_export_cnt","trade_export_val","trade_import_cnt","trade_import_val","trade_balance"]:
    tr[c] = pd.to_numeric(tr[c], errors="coerce").fillna(0)
tr["sido_norm"]    = tr["sido"].map(nrm)
tr["sigungu_norm"] = tr["sigungu"].map(nrm)
tr = tr.merge(map_trade, on=["sido_norm","sigungu_norm"], how="inner")
tr_agg = tr.groupby(["sgg_code10","yyyymm"], as_index=False)[["trade_export_cnt","trade_export_val","trade_import_cnt","trade_import_val","trade_balance"]].sum()

# Build full union month grid
keys = pd.concat([e9_agg[["sgg_code10","yyyymm"]],
                  pop_agg[["sgg_code10","yyyymm"]],
                  tr_agg[["sgg_code10","yyyymm"]]], ignore_index=True).drop_duplicates()

# Attach representative names & compute canonical (time-agnostic) names
keys = keys.merge(rep, on="sgg_code10", how="left")
keys[["canon_sido_name","canon_sigungu_name"]] = keys.apply(
    lambda r: pd.Series(canon_name_alltime(r["src_sido"], r["src_sigungu"])), axis=1
)
keys["canon_group_id"] = keys["canon_sido_name"].fillna("") + "::" + keys["canon_sigungu_name"].fillna("")
keys.loc[keys["canon_group_id"]=="::","canon_group_id"] = np.nan

# Aggregate each panel to canonical groups on the full union grid
def to_canon_full(df: pd.DataFrame, value_cols: list[str]) -> pd.DataFrame:
    x = df.merge(keys[["sgg_code10","yyyymm","canon_group_id","canon_sido_name","canon_sigungu_name"]],
                 on=["sgg_code10","yyyymm"], how="right")
    for c in value_cols:
        x[c] = pd.to_numeric(x[c], errors="coerce").fillna(0)
    out = (x.groupby(["canon_group_id","canon_sido_name","canon_sigungu_name","yyyymm"], as_index=False)[value_cols].sum())
    return out

e9_canon  = to_canon_full(e9_agg,  ["e9"])
pop_canon = to_canon_full(pop_agg, ["population_total","population_male","population_female"])
tr_canon  = to_canon_full(tr_agg,  ["trade_export_cnt","trade_export_val","trade_import_cnt","trade_import_val","trade_balance"])

# Outer-join all canon panels
panel = e9_canon.merge(pop_canon, on=["canon_group_id","canon_sido_name","canon_sigungu_name","yyyymm"], how="outer")
panel = panel.merge(tr_canon, on=["canon_group_id","canon_sido_name","canon_sigungu_name","yyyymm"], how="outer")

# Order & save
ordered = [
    "canon_sido_name","canon_sigungu_name","canon_group_id","yyyymm",
    "e9",
    "population_total","population_male","population_female",
    "trade_export_cnt","trade_export_val","trade_import_cnt","trade_import_val","trade_balance",
]
for c in ordered:
    if c not in panel.columns:
        panel[c] = np.nan
panel = panel[ordered].sort_values(["canon_sido_name","canon_sigungu_name","yyyymm"]).reset_index(drop=True)
panel = panel[pd.to_numeric(panel["yyyymm"], errors="coerce") >= 200901].copy()

panel.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
panel.shape, panel.head(12)


((45933, 13),
    canon_sido_name canon_sigungu_name canon_group_id  yyyymm    e9  \
 12         강원특별자치도                강릉시   강원특별자치도::강릉시  200901   0.0   
 13         강원특별자치도                강릉시   강원특별자치도::강릉시  200902   0.0   
 14         강원특별자치도                강릉시   강원특별자치도::강릉시  200903   0.0   
 15         강원특별자치도                강릉시   강원특별자치도::강릉시  200904   0.0   
 16         강원특별자치도                강릉시   강원특별자치도::강릉시  200905   0.0   
 17         강원특별자치도                강릉시   강원특별자치도::강릉시  200906  83.0   
 18         강원특별자치도                강릉시   강원특별자치도::강릉시  200907   0.0   
 19         강원특별자치도                강릉시   강원특별자치도::강릉시  200908   0.0   
 20         강원특별자치도                강릉시   강원특별자치도::강릉시  200909   0.0   
 21         강원특별자치도                강릉시   강원특별자치도::강릉시  200910   0.0   
 22         강원특별자치도                강릉시   강원특별자치도::강릉시  200911   0.0   
 23         강원특별자치도                강릉시   강원특별자치도::강릉시  200912  94.0   
 
     population_total  population_male  population_female  tr

시군구코드 추가

In [37]:
# Read files, inspect structure, then add `sgg_code10` to panel_sgg_monthly.csv using matching_table_join_keys.csv
# - Robust to column name variants
# - Saves to /mnt/data/panel_sgg_monthly_with_sgg_code10.csv
# - Shows quick previews

import pandas as pd
import numpy as np
from pathlib import Path
import unicodedata, re
from typing import List, Optional

BASE = Path("/Users/kjh/Documents/geostat3_e9")
PATH_MT   = BASE / "matching_table_join_keys.csv"
PATH_PAN  = BASE / "panel_sgg_monthly.csv"
OUT_PATH  = BASE / "panel_sgg_monthly_with_sgg_code10.csv"


def nrm(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFC", str(s)).strip()
    s = re.sub(r"\s+","", s)
    s = re.sub(r"[()（）\[\]]","", s)
    return s


def pick_first(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    # try case-insensitive / stripped match
    normmap = {c.strip().lower(): c for c in df.columns}
    for c in candidates:
        key = c.strip().lower()
        if key in normmap:
            return normmap[key]
    return None


# 1) Load files ---------------------------------------------------------------
mt = pd.read_csv(PATH_MT, dtype=str).rename(columns=lambda x: x.strip())
panel = pd.read_csv(PATH_PAN, dtype=str).rename(columns=lambda x: x.strip())

# Preview structures
preview_mt_cols = list(mt.columns)
preview_pan_cols = list(panel.columns)

# 2) Build flexible (sido, sigungu) -> sgg_code10 map -------------------------
# Ensure sgg_code10 exists even if named differently
sgg_candidates = ["sgg_code10","sgg_code","sgg","sgg_code_10","sggcd10","sgg_cd10","sgg_cd"]
sgg_col = pick_first(mt, sgg_candidates)
if sgg_col is None:
    raise ValueError("matching_table_join_keys.csv에 sgg_code10(혹은 유사명)이 없습니다.")
if sgg_col != "sgg_code10":
    mt = mt.rename(columns={sgg_col: "sgg_code10"})
mt["sgg_code10"] = mt["sgg_code10"].astype(str)

# Candidate (sido, sigungu) pairs from different sources inside the matching table
pair_candidates = [
    ("sido_pop","sigungu_pop"),
    ("sido_e9","sigungu_e9"),
    ("sido_trade","sigungu_trade"),
    # Sometimes the matching table may already include a generic (sido, sigungu)
    ("sido","sigungu"),
    ("sido_name","sigungu_name"),
    ("src_sido","src_sigungu"),  # if previously constructed
]

maps = []
for cs, cg in pair_candidates:
    if cs in mt.columns and cg in mt.columns:
        tmp = mt[[cs, cg, "sgg_code10"]].dropna()
        if not tmp.empty:
            tmp = tmp.assign(
                sido_norm = tmp[cs].map(nrm),
                sigungu_norm = tmp[cg].map(nrm),
            )
            # prefer the most frequent code if duplicated
            g = (tmp.groupby(["sido_norm","sigungu_norm"])["sgg_code10"]
                   .agg(lambda s: s.value_counts().index[0]).reset_index())
            maps.append(g)

if not maps:
    raise ValueError("매칭 테이블에서 (sido, sigungu) 쌍을 찾을 수 없습니다. 사용 가능한 컬럼을 확인하세요.")

map_union = pd.concat(maps, ignore_index=True).drop_duplicates()
# If multiple sgg_code10 per pair slipped through, keep the majority vote
map_union = (map_union.groupby(["sido_norm","sigungu_norm"])["sgg_code10"]
                       .agg(lambda s: s.value_counts().index[0]).reset_index())

# 3) Detect (sido, sigungu) in panel and merge --------------------------------
sido_col = pick_first(panel, ["canon_sido_name","sido","sido_name","시도","SIDO","sido_nm"])
sigungu_col = pick_first(panel, ["canon_sigungu_name","sigungu","sigungu_name","시군구","SGG","sgg","sigungu_nm"])
if not all([sido_col, sigungu_col]):
    raise ValueError("panel_sgg_monthly.csv에서 (sido, sigungu) 컬럼을 찾지 못했습니다.")

panel["_sido_norm"]    = panel[sido_col].map(nrm)
panel["_sigungu_norm"] = panel[sigungu_col].map(nrm)

panel2 = panel.merge(
    map_union.rename(columns={"sido_norm":"_sido_norm","sigungu_norm":"_sigungu_norm"}),
    on=["_sido_norm","_sigungu_norm"],
    how="left"
)

# 4) Save & show quick checks --------------------------------------------------
panel2.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
panel2.shape, panel2.head(12), preview_mt_cols, preview_pan_cols, OUT_PATH


((45933, 16),
    canon_sido_name canon_sigungu_name canon_group_id  yyyymm    e9  \
 0          강원특별자치도                강릉시   강원특별자치도::강릉시  200901   0.0   
 1          강원특별자치도                강릉시   강원특별자치도::강릉시  200902   0.0   
 2          강원특별자치도                강릉시   강원특별자치도::강릉시  200903   0.0   
 3          강원특별자치도                강릉시   강원특별자치도::강릉시  200904   0.0   
 4          강원특별자치도                강릉시   강원특별자치도::강릉시  200905   0.0   
 5          강원특별자치도                강릉시   강원특별자치도::강릉시  200906  83.0   
 6          강원특별자치도                강릉시   강원특별자치도::강릉시  200907   0.0   
 7          강원특별자치도                강릉시   강원특별자치도::강릉시  200908   0.0   
 8          강원특별자치도                강릉시   강원특별자치도::강릉시  200909   0.0   
 9          강원특별자치도                강릉시   강원특별자치도::강릉시  200910   0.0   
 10         강원특별자치도                강릉시   강원특별자치도::강릉시  200911   0.0   
 11         강원특별자치도                강릉시   강원특별자치도::강릉시  200912  94.0   
 
    population_total population_male population_female trade_

In [38]:
overrides = {
    "창원시": "48120",
    # 필요시 여기 추가: "청주시": "43110", "당진시": "44270", ...
}
panel2["sgg_code10"] = panel2["sgg_code10"].fillna(panel2["_sigungu_norm"].map(overrides))
panel2.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")


In [41]:
import pandas as pd
from pathlib import Path

BASE = Path("/Users/kjh/Documents/geostat3_e9/")

# --- 1. panel2는 이미 메모리에 있다고 가정 (없으면 아래 주석 해제해서 로드)
# panel2 = pd.read_csv(BASE / "panel_sgg_monthly_with_sgg_code10.csv", dtype=str)

# --- 2. Transform 시트 읽기
adm = pd.read_excel(
    BASE / "adm_code_general_2024.xlsx",
    sheet_name="Transform",
    dtype=str
)

# 필요한 열만 추출
adm_sub = adm[["SGGCD_ADM_2024", "IMSI_SDCD", "SGIS_2024"]].drop_duplicates()

# --- 3. join
panel2 = panel2.merge(
    adm_sub,
    left_on="sgg_code10",
    right_on="SGGCD_ADM_2024",
    how="left"
)

# SGIS_2024 앞 5자리만 사용
panel2["SGIS_2024"] = panel2["SGIS_2024"].astype(str).str.slice(0, 5)

# 컬럼명 변경
panel2 = panel2.rename(columns={
    "IMSI_SDCD": "sd_code",
    "SGIS_2024": "sgg_code_sgis"
})

# --- 4. panel3로 복사 + 중복 컬럼 제거
panel3 = panel2.loc[:, ~panel2.columns.duplicated()].copy()

# 혹시 그래도 'sgg_code_sgis'가 여러 개 남아 있을 경우 첫 번째 것만 사용
sgis_col = panel3["sgg_code_sgis"]
if isinstance(sgis_col, pd.DataFrame):           # 동일 이름 컬럼이 여러 개이면 DataFrame
    sgis_col = sgis_col.iloc[:, 0]               # 첫 번째 열만 사용
sgis_col = sgis_col.astype(str)

overrides = {
    "창원시": "38110",
    # "청주시": "43110",
    # "당진시": "44270",
    # 필요시 추가
}

# override용 Series
override_series = panel3["_sigungu_norm"].map(overrides)

# ① SGIS 코드가 있으면 그대로 사용, 없으면 overrides 사용해서 새 코드 생성
panel3["sgg_code_sgis"] = sgis_col.where(sgis_col.notna() & (sgis_col != "nan"), override_series)

# --- 5. 필요없는 키 컬럼 정리
panel3 = panel3.drop(columns=["SGGCD_ADM_2024, sgg_code10_from_sgis"], errors="ignore")

# --- 6. 저장
OUT_PATH = BASE / "panel_full.csv"
panel3.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

panel3.shape, OUT_PATH


((45933, 20), PosixPath('/Users/kjh/Documents/geostat3_e9/panel_full.csv'))

In [40]:
print(panel3.columns.tolist())

['canon_sido_name', 'canon_sigungu_name', 'canon_group_id', 'yyyymm', 'e9', 'population_total', 'population_male', 'population_female', 'trade_export_cnt', 'trade_export_val', 'trade_import_cnt', 'trade_import_val', 'trade_balance', '_sido_norm', '_sigungu_norm', 'sgg_code10', 'SGGCD_ADM_2024', 'sd_code', 'sgg_code_sgis']
